In [1]:
import getpass
import os

from langchain_core.tools import tool

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
            model="qwen-turbo",
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
            max_tokens=500,
            temperature=0.7,
        )



In [2]:
# build tools

@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

In [3]:
# chains

llm_with_tools = llm.bind_tools([multiply])
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_13759274dd214485960e8d',
  'type': 'tool_call'}]

In [4]:
chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
res = chain.invoke("What's four times 23")
print(res)

92


In [5]:
# agents

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# The agent prompt must have an agent_scratchpad key that is a
# MessagesPlaceholder. Intermediate agent actions and tool output messages will be passed in here.

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])


tools = [multiply, add, exponentiate]

# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, template)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke(
    {
        "input": "Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result"
    }
)



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 3, 'exponent': 5}`


243
Invoking: `add` with `{'first_int': 12, 'second_int': 3}`


15
Invoking: `multiply` with `{'first_int': 243, 'second_int': 15}`


3645
Invoking: `multiply` with `{'first_int': 3645, 'second_int': 3645}`


13286025Taking 3 to the fifth power yields 243. The sum of twelve and three is 15. Multiplying 243 by 15 gives us 3645. Squaring this result (multiplying it by itself) we get 13,286,025. 

Therefore, the final answer is 13,286,025.

> Finished chain.


{'input': 'Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result',
 'output': 'Taking 3 to the fifth power yields 243. The sum of twelve and three is 15. Multiplying 243 by 15 gives us 3645. Squaring this result (multiplying it by itself) we get 13,286,025. \n\nTherefore, the final answer is 13,286,025.'}